In [16]:
params = {}

In [17]:
import time
import json
import sys

from services.dte_tools.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, get_temp_dir, get_data_dir  # noqa: E402

In [18]:
   
environment = initialise_data_task("Jupyter Sample Data Upload Task", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))

if not params:
    environment.log_error("No parameters given!")
    

[2025-02-24T11:53:55.549085] Jupyter Sample Data Upload Task - INFO: Jupyter Sample Data Upload Task initialised running in active mode
Jupyter Sample Data Upload Task >>>: Jupyter Sample Data Upload Task initialised running in active mode


In [19]:
assert_dte_tools_available()

resolved_parameters = get_resolved_parameters_for_connection("ANA")
resolved_server_api_parameters = get_resolved_parameters_for_connection("DUFT-SERVER-API")
tmp_dir = get_temp_dir()
environment.log_message('Data Upload Starting!')
environment.log_message("Using parameters: %s" % resolved_parameters)
environment.log_message("Using temp directory: %s" % tmp_dir)
    
print(resolved_server_api_parameters)

DTE Tools available
[2025-02-24T11:53:55.555470] Jupyter Sample Data Upload Task - INFO: Data Upload Starting!
Jupyter Sample Data Upload Task >>>: Data Upload Starting!
[2025-02-24T11:53:55.556126] Jupyter Sample Data Upload Task - INFO: Using parameters: {'type': 'sqlite3', 'server': '127.0.0.1', 'sqlite3file': 'wakanda.sqlite', 'username': 'user', 'password': 'mysecretword', 'port': '5432', 'database': 'analysis'}
Jupyter Sample Data Upload Task >>>: Using parameters: {'type': 'sqlite3', 'server': '127.0.0.1', 'sqlite3file': 'wakanda.sqlite', 'username': 'user', 'password': 'mysecretword', 'port': '5432', 'database': 'analysis'}
[2025-02-24T11:53:55.556596] Jupyter Sample Data Upload Task - INFO: Using temp directory: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks
Jupyter Sample Data Upload Task >>>: Using temp directory: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks
{'server': 'http://127.0.0.1:8000'

In [20]:
# Load sample dataset from SQL File

import pandas as pd
import sqlite3
import os

# Load the data
conn = sqlite3.connect(os.path.join(get_data_dir(), resolved_parameters['sqlite3file']))
data = pd.read_sql_query("SELECT * FROM dim_client LIMIT 20", conn)
conn.close()

file_to_save = os.path.join(tmp_dir, 'dim_client_sample.csv')
data.to_csv(file_to_save, index=False)

data.head()



,client_id,emr_identifier,patient_pkv,facility_number,pbs_scenario,his_entry_type,number_h,birth_date,gender,death_date,...,is_disabled,disability,residence_ward,residence_village,residence_sub_county,residence_district,residence_county,residence_region,residence_country,date_imported_emr
0,1025682,1111130,,1025698,,EMR,,24/12/1960,Female,,...,,,,,,,,,Wakanda,
1,1129715,1215163,,1129731,,EMR,,9/4/1982,Female,,...,,,,,,,,,Wakanda,
2,1132571,1218019,,1132587,,EMR,,15/6/1983,Female,,...,,,,,,,,,Wakanda,
3,1132573,1218021,,1132589,,EMR,,14/10/1979,Male,,...,,,,,,,,,Wakanda,
4,1132575,1218023,,1132591,,EMR,,8/10/2008,Male,,...,,,,,,,,,Wakanda,


In [21]:
try:
    post_file_to_duft_server(
        resolved_server_api_parameters["server"], 
        resolved_server_api_parameters["username"], 
        resolved_server_api_parameters["password"], 
        file_to_save, 
        "client-list")
    
    environment.log_message('Data Upload Complete!')

except Exception as e:
    environment.log_error(f"Failed to upload file: {e}")



[2025-02-24T11:53:55.581050] Jupyter Sample Data Upload Task - ERROR: Failed to upload file: name 'post_file_to_duft_server' is not defined
Jupyter Sample Data Upload Task >>>: ERROR: Failed to upload file: name 'post_file_to_duft_server' is not defined


In [22]:
if os.path.exists(file_to_save):
    os.remove(file_to_save)
    environment.log_message(f"Removing data file: {file_to_save}")
else:
    environment.log_message(f"File does not exist: {file_to_save}")

[2025-02-24T11:53:55.585466] Jupyter Sample Data Upload Task - INFO: Removing data file: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks/dim_client_sample.csv
Jupyter Sample Data Upload Task >>>: Removing data file: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks/dim_client_sample.csv


In [23]:

environment.log_message('Data Upload Completed!')

[2025-02-24T11:53:55.588916] Jupyter Sample Data Upload Task - INFO: Data Upload Completed!
Jupyter Sample Data Upload Task >>>: Data Upload Completed!
